In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
def to_coordinates(number, N):
    coordinates = (number%N, math.floor(number/N))
    return coordinates

def to_tile_number(x, y, N):
    return x + y*N

In [ ]:
class Solution:
    
    def __init__(self, pieces):
        self.pieces = sorted(pieces)
        
    def N(self):
        return len(self.pieces)
    
    def is_fundamentally_different(self, others):
        for other in others:
            if self.is_transformation_of(other):
                return False
        return True
    
    def is_transformation_of(self, other):
        transformations = [rotation, double_rotation, triple_rotation,
                           mirror, mirror_rotation, 
                           mirror_double_rotation, mirror_triple_rotation]
        for transformation in transformations:
            if self == other.transform(transformation):
                return True
        return False
    
    def transform(self, transformation):
        new_pieces = [transformation(self.N(), piece) for piece in self.pieces]
        return Solution(new_pieces)
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.pieces == other.pieces
        else:
            return False
    
    # __str__ would be more appropriate, but [] redirects str() to repr()
    def __repr__(self):
        return 'Solution(' + str(self.pieces) + ')'

In [ ]:
def show_grid(solution):
    N = solution.N()
    
    # Make a NxN grid...
    image = np.zeros(N*N)

    # Set every 'other' cell to one
    whiteCells = [x+y*N for y in range(0, N) for x in range(y%2, N, 2)]
    image[whiteCells] = np.ones(N*N //2 + N%2)

    # show pieces
    image[solution.pieces] = np.ones(N) * 0.5
    
    image = image.reshape((N, N))
    row_labels = range(N)
    col_labels = [chr(65+i) for i in range(N)]
    plt.imshow(image)
    plt.xticks(range(N), col_labels)
    plt.yticks(range(N), row_labels)

In [ ]:
def queen_covers(N, queen, spot):
    (x_queen, y_queen) = to_coordinates(queen, N)
    (x_spot, y_spot) = to_coordinates(spot, N)
    if x_queen == x_spot:
        return True
    if abs(x_queen - x_spot) == abs(y_queen - y_spot):
        return True
    return False

def spot_free(N, queens, spot):
    for queen in queens:
        if queen_covers(N, spot, queen):
            return False
    return True

In [ ]:
def try_next_queen(N, queens, solutions):
    queen = len(queens)
    
    for column in range(N):
        queen_tile_number = queen*N + column
        
        if spot_free(N, queens, queen_tile_number):
            queens.extend([queen_tile_number])
            try_next_queen(N, queens, solutions)
            if len(queens) >= N:
                solutions.append(Solution(queens[:]))
            queens.pop()
                        
def solve_queens_problem(N):
    solutions = []
    for column in range(N):
        queens = [column]
        try_next_queen(N, queens, solutions)
    return solutions

In [ ]:
def show_all_solutions(solutions):
    if len(solutions) == 0:
        print('No solutions where found.')
        return
    
    print('The solutions are: ' + str(solutions))
    
    grid_size = math.ceil(math.sqrt(len(solutions)))

    plt.figure(figsize=(3*grid_size, 3*grid_size))

    for i, solution in enumerate(solutions):
        plt.subplot(grid_size, grid_size, i+1)
        show_grid(solution)

    plt.show()

In [ ]:
def rotation(N, tile_number):
    (x, y) = to_coordinates(tile_number, N)
    return to_tile_number(N-1 - y, x, N)

def double_rotation(N, tile_number):
    return rotation(N, rotation(N, tile_number))

def triple_rotation(N, tile_number):
    return rotation(N, double_rotation(N, tile_number))
    
def mirror(N, tile_number):
    (x, y) = to_coordinates(tile_number, N)
    return to_tile_number(N-1 - x, y, N)

def mirror_rotation(N, tile_number):
    return rotation(N, mirror(N, tile_number))

def mirror_double_rotation(N, tile_number):
    return double_rotation(N, mirror(N, tile_number))

def mirror_triple_rotation(N, tile_number):
    return triple_rotation(N, mirror(N, tile_number))

In [ ]:
def remove_symmetries(solutions):
    filtered_solutions = []
    for solution in solutions:
        if solution.is_fundamentally_different(filtered_solutions):
            filtered_solutions.append(solution)
    return filtered_solutions

In [ ]:
def run():
    for N in range(9):
        print('for N = '+ str(N))
        all_solutions = solve_queens_problem(N)
        different_solutions = remove_symmetries(all_solutions)
        show_all_solutions(different_solutions)
        
run()